In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

# Load library data

In [ ]:
file_path = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss63_09Bparams_2023-06-22_uniform-09b_n500_r100_f40_l10'
hdf = h5py.File(file_path+'/sam_lib.hdf5', 'r')

print(hdf.keys())
sample_params = hdf['sample_params'][...]
hc_ss = hdf['hc_ss'][...]
hc_bg = hdf['hc_bg'][...]
sspar = hdf['sspar'][...]
bgpar = hdf['bgpar'][...]
fobs_gw_cents = hdf['fobs'][:]
hdf.close()

dur = 1/fobs_gw_cents[0]
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur, num=len(fobs_gw_cents))
nsamps, nfreqs, nreals, nloudest = [*hc_ss.shape]
print(f"{nsamps=}, {nfreqs=}, {nreals=}, {nloudest=}")


# Load Param Space

In [ ]:
psfile = np.load(file_path+'/PS_Uniform_09B.pspace.npz')
print(psfile.files)
param_names = psfile['param_names']
print(param_names)
lib_name = '%s v%s, %d samples' % (psfile['class_name'], psfile['librarian_version'], nsamps)
print(lib_name)
psfile.close()


# Load Detstats

In [ ]:
dsfile = np.load(file_path+'/detstats/psrs40_sigma7.37e-06/detstats.npz')
print(dsfile.files)
dp_ss = dsfile['dp_ss']
dp_bg = dsfile['dp_bg']
df_ss = dsfile['df_ss']
df_bg = dsfile['df_bg']
snr_ss = dsfile['snr_ss']
snr_bg = dsfile['snr_bg']
ev_ss = dsfile['ev_ss']
dsfile.close()

nskies = ev_ss.shape[-1]
print(f"{ev_ss.shape=}, {dp_bg.shape=}")

In [ ]:
ev_skymean = np.mean(ev_ss, axis=-1)
evrat = ev_skymean/dp_bg

In [ ]:
hard_time = sample_params[:,0]
argsort = np.argsort(hard_time)

xx = hard_time[argsort]
yy = ev_ss[argsort].reshape(nsamps, nreals*nskies)
# y2 = np.median(dprat, axis=-1)[argsort]

fig, ax = plot.figax(xlabel='$t_\mathrm{hard}$ [Gyr]', ylabel='Expectation Value for SS Detections')

plot.draw_med_conf(ax, xx, yy)
# ax.scatter(xx,yy, label='SS EV/ BG DP', alpha=0.5)
# ax.scatter(xx, y2, label='SS DP/BG DP', alpha=0.5)
# ax.legend()

In [ ]:
hard_time = sample_params[:,0]
argsort = np.argsort(hard_time)

xx = hard_time[argsort]
yy = evrat[argsort]
# y2 = np.median(dprat, axis=-1)[argsort]

fig, ax = plot.figax(xlabel='$t_\mathrm{hard}$ [Gyr]', ylabel='exp # of SS / DP of BG')

plot.draw_med_conf(ax, xx, yy)
# ax.scatter(xx,yy, label='SS EV/ BG DP', alpha=0.5)
# ax.scatter(xx, y2, label='SS DP/BG DP', alpha=0.5)
# ax.legend()

In [ ]:
fig, ax = plot.figax(xlabel='hard_time', ylabel='DP')
xx = hard_time[argsort]